In [1]:
from openbabel import openbabel

In [2]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [4]:
FormoseAmmRels = './Rels/FormoseAmm_rels.tsv'
FormoseRels = './Rels/Formose_rels.tsv'
GlucoseAmmRels = './Rels/GlucoseAmm_rels.tsv'
GlucoseRels = './Rels/Glucose_rels.tsv'
PyruvicAcidRels = './Rels/PyruvicAcid_rels.tsv'

In [5]:
def indices(lst, element):
    result = []
    offset = -1
    while True:
        try:
            offset = lst.index(element, offset+1)
        except ValueError:
            return result
        result.append(offset)

In [6]:
def ProcessRels(path, title):
    Rels_data = pd.read_csv(path, sep='\t')
    indexes = []
    rules = []
    for i in range(len(Rels_data['Index'])):
        if Rels_data['Index'][i] not in indexes:
            indexes.append(Rels_data['Index'][i])
            rules.append(Rels_data['Rule'][i])
        if i%10000 == 0:
            print(i)
    print('Checkpoint')
    reactions = []
    reagents = []
    products = []
    for i in range(len(Rels_data['Index'])):
        reactions.append(Rels_data['Index'][i])
    print('Checkpoint 2')
    print(len(indexes))
    for i in range(len(indexes)):
        if i%5000 == 0:
            print(i)
        molecules = []
        coefficients = []
        dummy = indices(reactions, indexes[i])
        for j in range(len(dummy)):
            molecules.append(Rels_data['Reagent'][dummy[j]])
            coefficients.append(Rels_data['Formed/Produced'][dummy[j]])
        used = []
        made = []
        for k in range(len(coefficients)):
            if coefficients[k] == 1:
                made.append(molecules[k])
            else:
                used.append(molecules[k])
        reagents.append(used)
        products.append(made)
    data = {'Index': indexes, 'Reagents': reagents, 'Products': products, 'Rule': rules}
    df = pd.DataFrame(data)
    df.to_csv(f'{title}ProcessedRels.tsv', header=None, index=None, sep='\t', mode='a')
    return(df)

In [44]:
%%time
b = ProcessRels('./Rels/PyruvicAcid/PyruvicAcidRels_5.tsv', 'PyruvicAcid_5')

0
10000
20000
30000
40000
50000
Checkpoint
Checkpoint 2
17446
0
5000
10000
15000
CPU times: user 24.4 s, sys: 118 ms, total: 24.6 s
Wall time: 24.8 s


In [45]:
b

,Index,Reagents,Products,Rule
0,3_0,"[C(C(C)=O)(O)=O, C(C(C)=O)(O)=O]",[C(C(C)(CC(C(O)=O)=O)O)(O)=O],Aldol Condensation
1,5_0,"[C(C(C)=O)(O)=O, O]",[C(C(CO)O)(O)=O],Hydration of C(=O)C
2,8_0,[C(C(C)=O)(O)=O],"[C(C)=O, C(=O)=O]",Alpha-Keto Acid Decarboxylation
3,10_0,"[C(C)=O, C(C)=O]",[C(CC(C)O)=O],Aldol Condensation
4,12_0,"[C(C)=O, C(=O)=O]",[C(CC(O)=O)=O],Aldol Condensation
...,...,...,...,...
17441,23687_0,[C(C(CC(CO)O)CC=O)(O)=O],"[C(C=CC(O)=O)=O, CC(CO)O]","Michael Addition 0,2, (reverse)"
17442,23688_0,[C(C(C)(C(CO)O)CC=O)(O)=O],"[C(CO)O, C(C=C(C(O)=O)C)=O]","Michael Addition 0,2, (reverse)"
17443,23690_0,[C(C(C)(C(CO)O)CC=O)(O)=O],"[C, C(C(C(CO)O)=CC=O)(O)=O]","Michael Addition 0,2, (reverse)"
17444,23692_0,[C(C(C)C(C=O)C(CO)O)(O)=O],"[C, C(C=C(C=O)C(CO)O)(O)=O]","Michael Addition 0,2, (reverse)"
